## 나주시 빛가람동 법정동코드
### 4617013400

##### https://www.code.go.kr/stdcode/regCodeL.do

In [65]:
# 패키지 임포트
import requests
import pandas as pd
import json
import html
import xmltodict
import re

import warnings
warnings.filterwarnings('ignore')

In [67]:
# 경위도 좌표치환 메서드

def dms_to_decimal(degrees, minutes, seconds):
    return degrees + (minutes / 60) + (seconds / 3600)

# 문자열에서 도, 분, 초를 추출하는 함수
def parse_dms(dms_str):
    # 정규식을 이용해 DMS 문자열에서 도, 분, 초를 추출
    match = re.match(r"(\d+)° (\d+)' (\d+\.?\d*)\"", dms_str)
    if match:
        degrees = int(match.group(1))
        minutes = int(match.group(2))
        seconds = float(match.group(3))
        return dms_to_decimal(degrees, minutes, seconds)
    else:
        return None

# 라디오 채널 정보 API
##### 사용자가 주소 또는 자신의 위치 정보를 통해 해당 지역에서 청취가 가능한 라디오 채널정보(AM,FM) 정보 및 주파수 정보를 제공 받을 수 있습니다.

##### https://spectrummap.kr/cop/bbs/selectBoardArticle.do?menuNo=300486&bbsId=BBSMSTR_000000000172&nttId=3841&menuNo=300486&gubun=1

In [44]:
# 라디오 채널 정보 API
url = 'https://spectrummap.kr/openapi.do?'

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

key = ''
params = {
    'key': key, 
    'searchId': '02', # 검색 구분
    'ltd': '35.0184', # 경도
    'lou': '126.7891', # 위도
    'jusocode': '4617013400' # 법정동코드
}

response = requests.get(url, params=params, headers=headers)
xml_str = response.content.decode('utf-8')
xml_str = html.unescape(xml_str).lstrip("\r\n\t ").rstrip("\r\n\t ")
xml_str = xmltodict.parse(xml_str)['SpectrummapResponse']['row']

df = pd.DataFrame(xml_str)
df = df.rename(columns={
    'BROADAREA': '지역',
    'BROAD_NM': '방송사명',
    'SERVICE_NM': '채널명',
    'BROD_FREQ': '주파수'
})

In [58]:
df

,지역,방송사명,채널명,주파수
0,나주시,KBS,KBS 1라디오,747000
1,나주시,KBS,KBS 제2FM,95500000
2,나주시,MBC,MBC FM4U,91500000
3,나주시,광주영어방송,영어방송 FM,98700000
4,나주시,PBS,평화방송 FM,99900000
5,나주시,CBS,CBS 라디오,103100000
6,나주시,원음방송,원음방송 FM,107900000
7,나주시,BBS,불교방송 FM,89700000
8,나주시,나주소출력 FM,소출력FM,96100000
9,나주시,MBC,MBC 표준FM,93900000


# ============================

# 마을방송 무선국 정보 API (단위 : MHz)
##### 사용자가 주소 정보를 통해 해당 지역에서 청취 가능한 마을방송 무선국 정보 및 주파수 정보를 제공 받을 수 있습니다.
##### https://spectrummap.kr/cop/bbs/selectBoardArticle.do?menuNo=300543&bbsId=BBSMSTR_000000000172&nttId=14708&menuNo=300543&gubun=1

In [140]:
# 마을방송 무선국 정보 API
url = 'https://spectrummap.kr/openapiNew.do?'

key = ''

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params = {
    'key': key,  # 인증키 / 필수값
    'searchId': '05', # 호출서비스구분 / 필수값
    'type' : 'json', # 데이터타입
    #'pIndex': '', # 페이지위치 
    #'pSize': '', # 페이지당요청숫자
    'SIDO' : '46', # 시도코드 2자리 / 필수값
    'SIGUN' : '170', # 시군구코드 3자리 / 필수값
    'QUERY': 'ALL' # 주파수구분 - D1~D10, A1~A5, ALL
}

response = requests.get(url, params=params, headers=headers)
json_str = response.content.decode('utf-8')
json_object = json.loads(json_str)
df = pd.DataFrame(json_object['RESULT'])

In [142]:
# 컬럼명 변경 (기존 컬럼명 -> 새로운 컬럼명)
df = df.rename(columns={
    'CUS_FLN': '시설자명',
    'FRQ_HZ': '주파수(MHz)',
    'RNUM': '번호',
    'RDS_PMS_NO': '허가번호',
    'RDS_CLL_NM': '호출명칭',
    'EMDG_CD': '전파형식코드',
    'LON': '경도',
    'RDS_TRS_ADR': '주소(설치장소)',
    'LAT': '위도'
})

# 컬럼 순서 변경 (배치 순서대로 변경)
new_column_order = [
    '허가번호',        # 1
    '호출명칭',        # 2
    '시설자명',        # 3
    '전파형식코드',    # 4
    '주파수(MHz)',          # 5
    '주소(설치장소)',  # 6
    '경도',            # 7
    '위도'             # 8
]

# 데이터프레임을 새로운 컬럼 순서로 재정렬
df = df[new_column_order]

In [144]:
df['경도좌표'] = df['경도'].apply(parse_dms)
df['위도좌표'] = df['위도'].apply(parse_dms)

In [146]:
df

,허가번호,호출명칭,시설자명,전파형식코드,주파수,주소(설치장소),경도,위도,경도좌표,위도좌표
0,942022300000194,다시정가마을500,전남나주시다시면사무소,4K00F1E,422.034375,"전라남도 나주시 다시면 정가길 11, 정가마을회관 (가흥리)","126° 38' 00.13""","34° 59' 48.50""",126.633369,34.996806
1,942018300000259,산포신도마을100,전남나주시산포면사무소,4K00F1E,422.034375,"전라남도 나주시 산포면 신도길 103-7, 신도리1구마을회관 (신도리)","126° 48' 01.18""","35° 00' 24.44""",126.800328,35.006789
2,942018300000401,동강진천마을100,전남나주시동강면사무소,4K00F1E,422.021875,"전라남도 나주시 동강면 진천길 152, 진천2리회관 (진천리)","126° 34' 30.63""","34° 55' 30.59""",126.575175,34.925164
3,942022300000191,다시동산마을500,전남나주시다시면사무소,4K00F1E,422.021875,"전라남도 나주시 다시면 신석동산길 17, 동산마을회관 (신석리)","126° 37' 32.11""","34° 59' 10.55""",126.625586,34.986264
4,942022300000195,다시신흥마을500,전남나주시다시면사무소,4K00F1E,422.040625,"전라남도 나주시 다시면 무숙로 504, 신흥마을회관 (가흥리)","126° 38' 11.22""","34° 59' 30.57""",126.636450,34.991825
5,942017300000305,문평금옥마을300,전남나주시문평면사무소,8K50F3E,422.000000,"전라남도 나주시 문평면 금옥길 10, 금옥마을회관 (옥당리)","126° 37' 30.92""","35° 01' 32.79""",126.625256,35.025775
6,942017300000033,문평산호2마을300,전남나주시문평면사무소,8K50F3E,422.012500,"전라남도 나주시 문평면 산호평산길 64-1, 평산회관 2호 산호리(산호리)","126° 36' 13.72""","35° 02' 01.98""",126.603811,35.033883
7,942022300000189,다시초동마을500,전남나주시다시면사무소,4K00F1E,422.009375,"전라남도 나주시 다시면 초동길 40-2, 영촌마을회관 (영동리)","126° 37' 55.99""","35° 00' 03.86""",126.632219,35.001072
8,942014300000113,경연나주1호,전남나주시문평면사무소,8K50F3E,422.000000,"전라남도 나주시 문평면 산호남산길 17-20, 남산마을회관 (산호리)","126° 36' 22.91""","35° 02' 55.12""",126.606364,35.048644
9,942020300000628,도암행산마을500,전남화순군도암면사무소,4K00F1E,421.996875,"전라남도 나주시 다도면 행산길 8-4, (도동리)","126° 49' 44.00""","34° 52' 31.00""",126.828889,34.875278


# =====================

# 산악지역 이동통신 API - 데이터 많음 / 지역을 꼭 지정하길
##### 산악지역(국립공원, 도립공원, 군립공원) 또는 자신의 위치정보를 통해 해당지역에 위치한 이동통신서비스를 목적으로 개설된 무선국 정보를 제공 받을 수 있습니다.
##### https://spectrummap.kr/cop/bbs/selectBoardArticle.do?menuNo=300588&bbsId=BBSMSTR_000000000172&nttId=14341&menuNo=300510&gubun=1

In [85]:
# 산악지역 이동통신 API
url = 'https://spectrummap.kr/openapiNew.do?'

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

key = ''
params = {
    'key': key,  # 인증키 / 필수값
    'searchId': '07', # 호출서비스구분 / 필수값
    'type':'json', # 데이터타입
    'pIndex': '1', # 페이지위치 
    'pSize': '1000', # 페이지당요청숫자
    'SCH_CD' : 'MOBILE', # 검색구분 : MOBILE(기지국정보) / 필수값
    'PARK_CD' : '1', # 공원구분 : 1(국립공원), 2(도립공원), 3(군립공원) / 필수값
    'QUERY': '북한산', # 공원명 : 공원명 or ALL / 필수값
    'CUS_CD' : 'ALL', # 시설자 구분 : SK, KT, LG, ALL
    'SERVICE_CD' : 'ALL', # 서비스 구분 : 2G, 3G, 4G, 5G, ALL
    #'RSCP_PW' : 'ALL' # 
}

response = requests.get(url, params=params, headers=headers)
json_str = response.content.decode('utf-8')
json_object = json.loads(json_str)

In [89]:
df = pd.DataFrame(json_object['RESULT'])

In [91]:
# 컬럼명 변경 매핑
column_mapping = {
    'SERVICE_NAME': '용도',
    'RNUM': '번호',
    'EQM_CNT': '장치수',
    'RDS_PMS_NO': '허가번호',
    'ARW_GAN_NMV': '출력(W)',
    'GND_ALTD_HET': '지상고',
    'EXPS_ALTD_HET': '노출고',
    'LON': '경도',
    'ALT_ALTD_HET': '해발고',
    'RDS_TRS_ADR': '주소(설치장소)',
    'LAT': '위도',
    'ARW_FORM_CD': '안테나설치형태'
}

# 컬럼명 변경
df = df.rename(columns=column_mapping)

# 컬럼 순서 변경 (숫자에 맞게 정렬)
desired_order = [
    '허가번호',         # 1
    '주소(설치장소)',   # 2
    '용도',             # 3
    '장치수',           # 4
    '경도',             # 5
    '위도',             # 6
    '출력(W)',          # 7
    '안테나설치형태',    # 8
    '노출고',           # 9
    '지상고',           # 10
    '해발고'            # 11
]

# 컬럼 순서 변경
df = df[desired_order]

In [93]:
df['안테나설치형태'] = df['안테나설치형태'].apply(lambda x: str(x).strip('\r\n\t ') if isinstance(x, str) else x)

In [95]:
df

,허가번호,주소(설치장소),용도,장치수,경도,위도,출력,안테나설치형태,노출고,지상고,해발고
0,322003110000489,"서울특별시 강북구 삼양로173길 116-203, (우이동)",이동통신(IMT-2000)서비스 제공용(지상_광기지국),1,126.998533,37.665589,18.0,NaN,15.0,10.0,54.0
1,322003110000950,"서울특별시 강북구 삼양로173길 116-203, 나대지 (우이동)",이동통신(IMT-2000)서비스 제공용(지상_광기지국),1,126.998533,37.665589,17.5,강관주,30.0,30.0,100.0
2,322004110000843,서울 도봉구 도봉동 511,이동통신(IMT-2000)서비스 제공용(지상_광기지국),1,127.030262,37.687965,17.0,강관주,NaN,25.0,150.0
3,322006110001499,경기도 고양시 덕양구 효자동 산45 나대지,이동통신(IMT-2000)서비스 제공용(지상_광기지국),1,126.977133,37.675560,14.0,통신주(CP주),16.0,16.0,170.0
4,322006110001503,경기 고양시 덕양구 지축동 17-1,이동통신(IMT-2000)서비스 제공용(지상_광기지국),1,126.963054,37.689313,14.0,통신주(CP주),15.0,16.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...
995,322021110008821,"경기도 고양시 덕양구 대서문길 375, 부왕사부근 (북한동)",이동전화(5G-NR(3.5㎓))서비스 제공용(지상_기지국),0,126.970658,37.641575,22.3,NaN,25.0,25.0,392.0
996,322021110008823,"경기도 고양시 덕양구 대서문길 197-20, 시단봉부근 (북한동)",이동전화(5G-NR(3.5㎓))서비스 제공용(지상_기지국),0,126.983169,37.640422,22.3,NaN,14.0,14.0,184.0
997,322021110008865,"서울특별시 강북구 삼양로173길 116-203, 용덕사옆전주 (우이동)",이동전화(5G-NR(3.5㎓))서비스 제공용(지상_기지국),0,127.020092,37.670597,22.3,통신주(CP주),14.0,14.0,185.0
998,322021110008866,"서울특별시 강북구 삼양로173길 116-203, 우이동신검사 (우이동)",이동전화(5G-NR(3.5㎓))서비스 제공용(지상_기지국),0,127.000219,37.672881,22.3,NaN,2.0,3.0,235.0


# ===================

# 우량경보 무선국 정보

##### 사용자가 주소 정보를 통해 해당 지역에서 우량경보 무선국 정보 및 주파수 정보를 제공 받을 수 있습니다.
##### https://spectrummap.kr/cop/bbs/selectBoardArticle.do?menuNo=300615&bbsId=BBSMSTR_000000000172&nttId=15134&menuNo=300615&gubun=1

In [5]:
# 우량경보 무선국 정보 API
url = 'https://spectrummap.kr/openapiNew.do?'

# 요청 헤더에 User-Agent 추가
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

key = ''
params = {
    'key': key,  # 인증키 / 필수값
    'searchId': '08', # 호출서비스구분 / 필수값
    'type' : 'json', # 데이터타입
    'pIndex': '1', # 페이지위치 
    'pSize': '10', # 페이지당요청숫자
    'SIDO' : '48', # 시도코드 2자리 / 필수값
    'SIGUN' : '860' # 시군구코드 3자리 / 필수값
}

response = requests.get(url, params=params)
json_str = response.content.decode('utf-8')

In [11]:
json_object = json.loads(json_str)
df = pd.DataFrame(json_object['RESULT'])

In [21]:
# 컬럼명 변경 매핑
column_mapping = {
    'FRQ_HZ' : '주파수',
    'RNUM' : '번호',
    'RDS_PMS_NO' : '허가번호',
    'RDS_CLL_NM' : '우량국명',
    'LON' : '위도',
    'GUBUN' : '우량국종류',
    'RDS_TRS_ADR' : '주소(설치장소)', 
    'LAT' : '경도'
}

# 컬럼명 변경
df = df.rename(columns=column_mapping)

# 컬럼 순서 변경 (숫자에 맞게 정렬)
desired_order = [
    '허가번호',         # 1
    '우량국명',         # 2
    '우량국종류',       # 3
    '주소(설치장소)',   # 4
    '경도',            # 5
    '위도',            # 6
    '주파수'           # 7
]

# 컬럼 순서 변경
df = df[desired_order]

In [23]:
df

,허가번호,우량국명,우량국종류,주소(설치장소),경도,위도,주파수
0,212000200000020,법계사우량국,우량국,경상남도 산청군 시천면 중산리 산208번지 055-970-6713,35.289982,127.773130,159900000
1,212000200000021,문창대우량국,우량국,"경상남도 산청군 시천면 지리산대로 320-103, (중산리)",35.315383,127.745686,159900000
2,212000200000022,산청순두류우량국,우량국,경상남도 산청군 시천면 중산리 산92-2번지,35.319427,127.756947,159900000
3,212000200000023,중산1경보국,경보국,경상남도 산청군 시천면 중산리 479번지,35.294800,127.755700,159900000
4,212000200000024,삼장면통제국,통제국,"경상남도 산청군 삼장면 덕산대포로 257, (대포리)",35.277686,127.838081,159887500
5,212000200000026,외고개우량국,우량국,경상남도 산청군 삼장면 유평리 산42번지,35.381938,127.784221,159887500
6,212000200000027,대원교경보국,경보국,경상남도 산청군 삼장면 유평리 산4-1번지,35.354294,127.812737,159887500
7,212000200000028,산청노루목경보국,경보국,경상남도 산청군 삼장면 유평리 산3-3번지,35.355727,127.806836,159887500
8,212000200000029,중산4경보국,경보국,경상남도 산청군 시천면 중산리 산89-5번지,35.303694,127.749178,159900000
9,212002200000049,삼장장당제1경보,경보국,경상남도 산청군 삼장면 내원리 산146번지,35.310370,127.792252,159887500
